## Import packages

In [1]:
import os, tempfile, urllib
import pandas as pd
import tensorflow as tf

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))

TensorFlow version: 2.4.1


## Downloading data

In [3]:
data_filename = 'IMDB_dataset.csv'
data_url = 'https://raw.githubusercontent.com/dimitreOliveira/bert-as-a-service_TFX/main/Data/IMDB_5k_dataset.csv'

# Create data directory
_data_dir = os.path.join(tempfile.gettempdir(), 'data/')
os.mkdir(_data_dir)

# Create output directory
_output_dir = os.path.join(tempfile.gettempdir(), 'output/')
os.mkdir(_output_dir)

# Download data
urllib.request.urlretrieve(data_url, f'{_data_dir}{data_filename}')

('/tmp/data/IMDB_dataset.csv', <http.client.HTTPMessage at 0x7f3abea7aa10>)

Take a quick look at the data file.

In [4]:
data = pd.read_csv(_data_dir+data_filename)

display(data.head())

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### Parameters

In [5]:
num_samples = 5000

### Helper functions


In [6]:
def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def create_example(review, sentiment):
    feature = {
        'review': bytes_feature(review),
        'sentiment': int64_feature(sentiment),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def parse_tfrecord_fn(example):
    feature_description = {
        'review': tf.io.FixedLenFeature([], tf.string),
        'sentiment': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example


def generate_tfrecords(data, num_samples, output_dir):
    num_tfrecods = len(data) // num_samples
    if len(data) % num_samples:
        num_tfrecods += 1  # add one record if there are any remaining samples

    for tfrec_num in range(num_tfrecods):
        samples = data[(tfrec_num * num_samples) : ((tfrec_num + 1) * num_samples)]

        with tf.io.TFRecordWriter(output_dir + "/file_%.2i-%i.tfrec" % (tfrec_num, len(samples))) as writer:
            for row in samples.itertuples(index=False):
                example = create_example(row.review, row.sentiment)
                writer.write(example.SerializeToString())

## Generate data in the TFRecord format

In [7]:
generate_tfrecords(data, num_samples, _output_dir)

## Explore one sample from the generated TFRecord

In [8]:
raw_dataset = tf.data.TFRecordDataset(f"{_output_dir}/file_00-{num_samples}.tfrec")
parsed_dataset = raw_dataset.map(parse_tfrecord_fn)

for features in parsed_dataset.take(3):
    print(f"review: {features['review'].numpy().decode()}")
    print(f"sentiment: {features['sentiment']}\n")

review: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to